**Import Required Libraries and Set Up the Environment**

In this cell, we import necessary libraries from Qiskit, Quairkit, NumPy, and other dependencies. These imports include tools for building and manipulating quantum circuits, handling noise models, and utilizing IBM's quantum runtime service.



Before running this Jupyter Notebook, please ensure that the following Python packages are installed. You can use the following pip command to install these packages:

```sh
pip install quairkit qiskit qiskit_ibm_runtime qiskit_aer tqdm pylatexenc matplotlib
```

In [ ]:
import os

# Import essential Qiskit components to create and manipulate quantum circuits
# Type aliases for better readability and clarity of type annotations
from typing import Optional, Union

# Import NumPy for numerical operations
import numpy as np
from numpy import ndarray

# Import Qiskit components for building quantum circuits
from qiskit import QuantumCircuit, QuantumRegister, transpile
from qiskit.circuit.library import CXGate, U3Gate
from qiskit.quantum_info import Operator, Statevector

# Import Qiskit Aer components for simulation and noise modeling
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel

# Import the Qiskit Runtime Service, used for accessing IBM's quantum resources
from qiskit_ibm_runtime import QiskitRuntimeService

# Import the partial_trace and random_unitary functions from quairkit's quantum information tools
from quairkit.qinfo import partial_trace
from quairkit.database import random_unitary

# Import TQDM for creating progress bars in loops
from tqdm import tqdm


**Define and Decompose the FT3 Matrix Circuit**

In this cell, we define a unitary matrix called FT3, and create a quantum circuit that applies this matrix to two qubits. The circuit is then decomposed into fundamental gates that are commonly used on IBM Quantum devices, such as the CX and U3 gates. Finally, we visualize the decomposed version of the quantum circuit.


In [ ]:
# Define the FT3 matrix, which is a 4x4 unitary matrix with normalized elements
ft3 = np.array([[0, 1, 1, 1], 
                [1, 0, 1, -1], 
                [1, -1, 0, 1], 
                [1, 1, -1, 0]]) / np.sqrt(3)

# Create a quantum circuit with 2 qubits and apply the FT3 unitary matrix to them
ft3_matrix_qc = QuantumCircuit(2)
ft3_matrix_qc.unitary(Operator(ft3), [0, 1], label="ft3")

# Decompose the unitary matrix into basic quantum gates using Qiskit's transpile function
# The basis_gates parameter specifies the gates to which the unitary should be decomposed, suitable for IBM Quantum devices
decomposed_circuit = transpile(ft3_matrix_qc, basis_gates=["cx", "u3"])

# Draw and display the decomposed version of the quantum circuit using matplotlib
display(decomposed_circuit.draw("mpl"))


**Add Control Qubit to the Decomposed FT3 Circuit**

In this cell, we extend the decomposed FT3 quantum circuit by adding an auxiliary control qubit. This allows us to create a new version of the circuit where each original gate is controlled by the auxiliary qubit. For single-qubit gates, controlled versions are added, and for CNOT gates, a Toffoli (CCX) gate is used. Finally, the modified circuit is visualized.


In [ ]:
# Assuming 'decomposed_circuit' is your original decomposed circuit
# Create a new quantum register to add an auxiliary qubit for controlling the gates
aux_qr = QuantumRegister(1)
original_qubits = decomposed_circuit.qubits

# Initialize a new quantum circuit with the auxiliary qubit and the original qubits
ctrl_ft3_qc = QuantumCircuit(aux_qr, original_qubits)

# Apply the global phase of the original circuit to the new circuit to maintain consistency
ctrl_ft3_qc.p(decomposed_circuit.global_phase, 0)

# Iterate over each gate in the original decomposed circuit to create a controlled version
for gate, qubits, cbits in decomposed_circuit.data:
    # If the gate is a single-qubit U3 gate, add its controlled version to the new circuit
    if isinstance(gate, U3Gate):
        ctrl_gate = gate.control()  # Create a controlled version of the U3 gate
        # Append the controlled gate with the auxiliary qubit as the control
        ctrl_ft3_qc.append(ctrl_gate, [aux_qr[0]] + qubits, cbits)
    # If the gate is a CNOT (CX) gate, replace it with a Toffoli (CCX) gate
    elif isinstance(gate, CXGate):
        ctrl_ft3_qc.ccx(aux_qr[0], qubits[0], qubits[1])

# Draw and display the new quantum circuit with controlled gates using matplotlib
display(ctrl_ft3_qc.draw(output="mpl"))


**Construct a Controlled-Z (CZ) Circuit**

In this cell, we construct a quantum circuit with two qubits to demonstrate the use of the Controlled-Z (CZ) gate. First, we apply X gates to both qubits, effectively flipping their states. Next, we apply a CZ gate, with the first qubit acting as the control and the second as the target. Finally, we apply X gates again to return both qubits to their original states. This sequence provides an example of how CZ gates can be used in quantum algorithms, and the circuit is visualized at the end.


In [ ]:
# Initialize a quantum circuit with two qubits
cz_qc = QuantumCircuit(2)

# Step 1: Apply X gates to flip the state of both qubits from |0⟩ to |1⟩
cz_qc.x([0, 1])

# Step 2: Apply a Controlled-Z (CZ) gate with qubit 0 as control and qubit 1 as target
cz_qc.cz(0, 1)

# Step 3: Apply X gates again to both qubits to return them to their original state
cz_qc.x([0, 1])

# Optional: Draw the quantum circuit to visualize the sequence of operations
display(cz_qc.draw("mpl"))


**Construct a Three-Qubit Quantum Circuit with Controlled Operations**

In this cell, we create a quantum circuit involving three qubits, named "G2". The circuit includes various quantum gates applied to manipulate qubit states, such as Hadamard gates, Controlled-Z (CZ) gates, Toffoli gates, and other controlled operations. The circuit construction includes the following steps:
1. Apply Hadamard gates to the second and third qubits.
2. Add the previously constructed CZ circuit, targeting the second and third qubits.
3. Apply Hadamard gates again to the second and third qubits.
4. Apply X gates, a Toffoli gate (CCX), and then X gates again for specific state manipulation.
5. Finally, compose the controlled version of the FT3 circuit into the G2 circuit.
The complete quantum circuit is then visualized.


In [ ]:
# Create a quantum register with 3 qubits and initialize the quantum circuit named "G2"
G2_qr = QuantumRegister(3, name="g2_register")
G2_qc = QuantumCircuit(G2_qr, name="G2")

# Apply Hadamard gates to the second and third qubits to create superposition
G2_qc.h(G2_qr[1:3])

# Compose the previously defined CZ circuit, applying it to qubits 1 and 2
G2_qc.compose(cz_qc, [G2_qr[1], G2_qr[2]], inplace=True)

# Apply Hadamard gates again to the second and third qubits
G2_qc.h(G2_qr[1:3])

# Apply X gates to the second and third qubits
G2_qc.x(G2_qr[1:3])

# Apply a Toffoli gate (CCX) with qubits 2 and 1 as controls and qubit 0 as the target
G2_qc.ccx(G2_qr[2], G2_qr[1], G2_qr[0])

# Apply X gates again to the second and third qubits to revert their state
G2_qc.x(G2_qr[1:3])

# Compose the controlled FT3 circuit using all three qubits
G2_qc.compose(ctrl_ft3_qc, G2_qr[0:3], inplace=True)

# Draw and display the final quantum circuit
display(G2_qc.draw("mpl"))


**Create an Entangled Input State (Bell State)**

In [ ]:
# Create an entangled input state (Bell state)
# The state is an equal superposition of the two-qubit states |00⟩ and |11⟩
psi_in: Statevector = (
    Statevector.from_label("00") + Statevector.from_label("11")
) / np.sqrt(2)


**Create a Quantum Circuit with a Specified Unitary Operation**

In this cell, we define a function `create_circuit_with_unitary()` that constructs a quantum circuit involving five qubits. The function applies a given unitary operation to specific qubits within a complex sequence of operations, including initialization, Hadamard gates, and controlled gates. The process involves multiple iterations where controlled-X (CX) and controlled-Y (CY) gates are applied, as well as conditional compositions with previously defined circuits like `cz_qc` and `G2_qc`. This circuit construction demonstrates how custom unitary operations can be incorporated into a larger quantum system.

**Function Details**:
- **Input Parameters**:
  - `u`: A unitary operation represented as either a Qiskit `Operator` or a NumPy array (`ndarray`).
  - `psi_in_vector`: The initial state vector to be applied to qubits 3 and 4, used to create an entangled or specific quantum state.

- **Key Steps**:
  1. **Initialization**: The first three qubits are initialized to the |0⟩ state, and qubits 3 and 4 are initialized to the state represented by `psi_in_vector`.
  2. **Controlled Gates and Unitary Application**: A sequence of CX, CY, and unitary gates are applied to the fourth qubit, repeatedly in a loop.
  3. **Conditional Composition**: Based on the loop iteration, additional circuits like `cz_qc` and `G2_qc` are composed conditionally.
  4. **Final Gates and Return**: Additional Hadamard and controlled gates are applied at the end of the loop before returning the constructed quantum circuit.

This function provides a flexible way to create a complex quantum circuit with custom unitary operations, useful in quantum algorithm design and experimentation.


In [ ]:
def create_circuit_with_unitary(
    u: Union[Operator, ndarray], psi_in_vector: Statevector
) -> QuantumCircuit:
    """
    Create a quantum circuit with a given unitary operation applied to the fourth qubit.
    The circuit includes initialization, controlled gates, Hadamard gates,
    and additional controlled operations, and returns the final constructed circuit.

    Args:
        u: A unitary operation represented as either a Qiskit Operator object or NumPy ndarray.
        psi_in_vector: The initial state vector used to initialize qubits 3 and 4.

    Returns:
        The constructed QuantumCircuit object with the specified unitary operation.
    """
    # Create a quantum register and quantum circuit with five qubits
    qr = QuantumRegister(5)
    qc = QuantumCircuit(qr)

    # Initialize the first three qubits to the |0> state
    for i in range(3):
        qc.initialize([1, 0], i)

    # Initialize the fourth and fifth qubits to the provided statevector (entangled state)
    qc.initialize(psi_in_vector, [3, 4])

    # Apply Hadamard gates to the second and third qubits to create superposition
    qc.h(qr[1:3])

    # Loop to apply a sequence of gates, including the provided unitary operation 'u'
    for i in range(4):
        qc.cx(qr[2], qr[3])  # Apply a controlled-X gate from qubit 2 to qubit 3
        qc.cy(qr[1], qr[3])  # Apply a controlled-Y gate from qubit 1 to qubit 3
        qc.unitary(u, [qr[3]], label="U")  # Apply the unitary operation to qubit 3
        qc.cy(qr[1], qr[3])  # Apply another controlled-Y gate from qubit 1 to qubit 3
        qc.cx(qr[2], qr[3])  # Apply another controlled-X gate from qubit 2 to qubit 3

        # Conditionally compose additional circuits at certain loop iterations
        if i == 2:
            qc.compose(cz_qc, qubits=[qr[1], qr[2]], inplace=True)

        # Conditionally compose the G2 circuit or its inverse, based on the iteration index
        if i in [0, 2]:
            qc.compose(G2_qc, qubits=[qr[0], qr[1], qr[2]], inplace=True)
            qc.x(qr[0])  # Apply an X gate to qubit 0 for state alteration
        elif i in [1, 3]:
            qc.compose(G2_qc.inverse(), qubits=[qr[0], qr[1], qr[2]], inplace=True)

    # Apply final Hadamard gates to the second and third qubits
    qc.h(qr[1:3])
    
    # Apply controlled operations to the third and fourth qubits
    qc.cy(qr[2], qr[3])
    qc.cx(qr[1], qr[3])

    # Return the final quantum circuit
    return qc


**Simulate Quantum Circuit and Obtain Output Density Matrix**

In this cell, we define a function `get_output_density_matrix()` that simulates a given quantum circuit using a specified noise model and returns the resulting density matrix. This function is useful for analyzing the effects of noise on quantum circuits and understanding how the final state evolves under realistic conditions.

**Function Details**:
- **Input Parameters**:
  - `qc`: The `QuantumCircuit` object to be simulated.
  - `noise_model`: An optional `NoiseModel` to apply during the simulation. If no noise model is provided, the simulation will assume ideal conditions without noise.
  
- **Key Steps**:
  1. **Simulator Creation**: A noisy simulator (`AerSimulator`) is created using the provided noise model.
  2. **Circuit Transpilation**: The circuit is transpiled to match the requirements of the simulator.
  3. **Simulation Execution**: The circuit is simulated, and the density matrix is obtained as the output.
  4. **Return Values**: The function returns the output density matrix and the depth of the transpiled circuit.

This function is particularly valuable in assessing the impact of noise on quantum computations, which is critical in designing robust quantum algorithms.


In [ ]:
def get_output_density_matrix(
    qc: QuantumCircuit,
    noise_model: Optional[NoiseModel] = None,
) -> tuple:
    """
    Simulate a quantum circuit with the given noise model and return the final density matrix and circuit depth.

    Args:
        qc: The QuantumCircuit object to be simulated.
        noise_model: The noise model to apply during the simulation. If None, no noise will be applied.

    Returns:
        A tuple containing:
            - The final density matrix obtained after the simulation.
            - The depth of the transpiled circuit.
    """
    # Create a noisy simulator using the provided noise model (if any)
    sim_noise = AerSimulator(noise_model=noise_model)

    # Transpile the quantum circuit for the noisy simulator to optimize execution
    transpiled_circuit = transpile(qc, sim_noise)

    # Add an instruction to save the density matrix of the final quantum state
    transpiled_circuit.save_density_matrix(label="output_density_matrix")

    # Run the simulation with the noisy simulator
    simulation_result = sim_noise.run(transpiled_circuit).result()

    # Extract the output density matrix from the simulation results and reverse the qubit ordering
    output_density_matrix = simulation_result.data(0)["output_density_matrix"].reverse_qargs().data

    # Return the output density matrix and the depth of the transpiled circuit
    return output_density_matrix, transpiled_circuit.depth()


**Calculate Fidelity of a Quantum State After Unitary Transformation**

In this cell, we define a function `calculate_fidelity()` that calculates the fidelity between a given statevector and an expected final state after applying a unitary transformation to a quantum circuit. Fidelity is a measure of how close two quantum states are, with a value between 0 and 1, where 1 indicates perfect overlap.

**Function Details**:
- **Input Parameters**:
  - `output_state_density_matrix`: The density matrix representing the output state obtained from the simulation.
  - `u`: The unitary transformation matrix applied during the circuit.
  - `psi_in_vector`: The initial statevector before the circuit, representing the state that we use as reference.
  
- **Key Steps**:
  1. **Partial Trace Calculation**: The function computes a partial trace of the output density matrix to reduce it to the desired qubit set.
  2. **Fidelity Calculation**: Using the unitary transformation `u` and the initial state vector, the function calculates the fidelity between the transformed expected state and the simulated output state.

This function helps in evaluating the performance of a quantum algorithm by providing a measure of how well the final state matches the desired one after applying specific quantum operations.


In [ ]:
def calculate_fidelity(
    output_state_density_matrix: ndarray, u: ndarray, psi_in_vector: Statevector
) -> float:
    """
    Calculate the fidelity between a given output density matrix and an expected initial state
    after applying a unitary transformation.

    Args:
        output_state_density_matrix: The density matrix representing the output state from simulation.
        u: The unitary matrix applied during the circuit to transform the state.
        psi_in_vector: The initial statevector before the circuit that serves as a reference for fidelity calculation.

    Returns:
        The calculated fidelity as a float value, representing the closeness of the final output state to the expected state.
    """
    # Compute the partial trace of the output density matrix to isolate the desired subsystem
    output_psi_dm = partial_trace(output_state_density_matrix, [0, 1, 2]).data

    # Convert the initial statevector into a column vector for matrix calculations
    psi_in_ket = psi_in_vector.data.reshape([-1, 1])

    # Expand the unitary matrix to act on the extended system by taking a Kronecker product with the identity
    u = np.kron(u, np.eye(2))

    return (
        psi_in_ket.conj().T @ u @ output_psi_dm @ u.conj().T @ psi_in_ket
    ).real.item()


**Initialize IBM Quantum Service and Load Unitary Set**

In this cell, we initialize the IBM Quantum service to access the quantum backends available on the IBM Cloud. We specify the names of the available quantum devices (`ibm_osaka`, `ibm_brisbane`, and `ibm_sherbrooke`) to be used for executing our quantum circuit. The function also loads a set of precomputed unitary matrices, which will be used during the quantum experiments.

**Key Steps**:
1. **Backend Initialization**: Define a list of IBM Quantum backends that will be used to run the quantum experiments.
2. **Service Initialization**: Initialize the IBM Quantum service. Note that sensitive credentials like `token` and `instance` are placeholders and should be securely managed, such as using environment variables.
3. **Unitary Set Loading**: Load a set of random unitary matrices from a file, which will be used during the experiment. These unitary operations are generated beforehand and used as part of the circuit.
4. **Output File Preparation**: Prepare a filename for saving the results, including the noise data collected during the experiments.

**Security Note**: Avoid including sensitive information such as API tokens in publicly accessible scripts. Use environment variables or secure vaults to manage them securely.


In [ ]:
# Name of the quantum circuit to be used
circuit_name = "qft43"

# List of backend names representing the available IBM Quantum devices
backends = ["ibm_osaka", "ibm_brisbane", "ibm_sherbrooke"]

# Initialize the IBM Quantum service to access the quantum devices on IBM Cloud
# Ensure to replace 'your_ibm_cloud_token_here' and 'your_instance_here' with actual values in a secure manner
service = QiskitRuntimeService(
    channel="ibm_cloud",
    token="your_ibm_cloud_token_here",  # Replace with your IBM Quantum token
    instance="your_instance_here"       # Replace with your IBM Quantum instance string
)

# Set the number of unitary operations to be tested and load the unitary set from a file
num_test = 100
# Check if the unitary set file exists
unitary_file = f"{num_test}_random_unitaries.npy"
if os.path.exists(unitary_file):
    unitary_set = np.load(unitary_file)
else:
    unitary_set = random_unitary(num_test).numpy()
    np.save(unitary_file, unitary_set)

# Prepare a filename to save the noise table from the quantum experiments
table_file_name = f"ibm_noise_table_{num_test}.csv"


**Compute Fidelity Across Different IBM Quantum Backends**

In this cell, we evaluate the fidelity and depth of a quantum circuit executed on different IBM Quantum backends. By looping over a list of available backends, we simulate the circuit using backend-specific noise models and gather metrics on the performance of each backend.

**Key Steps**:
1. **Backend Loop**: Iterate over each backend to evaluate the performance.
2. **Noise Model Extraction**: For each backend, generate a noise model to simulate the quantum circuit under realistic conditions.
3. **Circuit Simulation and Fidelity Calculation**:
   - For each unitary in the preloaded unitary set, create a quantum circuit.
   - Simulate the circuit using the backend noise model.
   - Calculate the fidelity between the resulting quantum state and the expected state.
4. **Result Collection**:
   - Calculate the average fidelity and depth for each backend.
   - Store the results in lists and save them to a CSV file for future analysis.

This process helps in assessing the performance of different quantum backends when executing a specific quantum algorithm, providing insight into the best backend for specific use cases.


In [ ]:
# Initialize lists to store backend fidelities and depths
backend_fidelities = []
backend_depths = []

# Loop over each backend to compute the average fidelity and circuit depth
for backend_name in tqdm(backends):
    # Retrieve the backend using the IBM Quantum service
    backend = service.backend(backend_name)

    # Generate a noise model based on the characteristics of the backend
    noise_model = NoiseModel.from_backend(backend)

    fidelities = []  # To store fidelities for each unitary operation
    depths = []  # To store circuit depths for each unitary operation

    # Loop over each unitary in the precomputed unitary set
    for u in tqdm(unitary_set, desc=f"Processing unitary set for backend {backend_name}"):
        # Create a quantum circuit with the given unitary and initial state
        qc = create_circuit_with_unitary(u, psi_in)

        # Simulate the circuit with the noise model and retrieve the density matrix and circuit depth
        output_state_density_matrix, depth = get_output_density_matrix(qc, noise_model)

        # Calculate the fidelity of the resulting quantum state with respect to the expected state
        fidelity = calculate_fidelity(output_state_density_matrix, u, psi_in)

        # Append the calculated fidelity and circuit depth to the respective lists
        fidelities.append(fidelity)
        depths.append(depth)

    # Calculate the average fidelity and average circuit depth for the current backend
    average_fidelity = np.mean(fidelities)
    average_depth = np.mean(depths)

    # Store the average fidelity and depth for the current backend
    backend_fidelities.append(average_fidelity)
    backend_depths.append(average_depth)

    # Save the results of the current backend iteration
    qc_backend_fidelity = {
        "circuit_name": circuit_name,
        "backend_name": backend_name,
        "average_fidelity": average_fidelity,
        "average_depth": average_depth,
    }

    # Append the results to the CSV file, creating a new header if the file is empty
    with open(table_file_name, "a") as file:
        if file.tell() == 0:  # If the file is empty, write the header
            file.write("circuit_name,backend_name,average_fidelity,average_depth\n")
        # Write the backend results to the file
        file.write(
            f"{qc_backend_fidelity['circuit_name']},"
            f"{qc_backend_fidelity['backend_name']},"
            f"{qc_backend_fidelity['average_fidelity']},"
            f"{qc_backend_fidelity['average_depth']}\n"
        )


**Visualize the Quantum Circuit**

In this cell, we visualize the quantum circuit using Qiskit's built-in drawing capabilities. The output is generated as a Matplotlib figure (`mpl`), which allows for easy integration into reports and visual analysis. This visualization provides an intuitive way to understand the quantum gates and their arrangement within the circuit.


In [ ]:
# Visualize the quantum circuit using Qiskit's Matplotlib drawer
qc.draw(output="mpl")


**Calculate the Depth of the Quantum Circuit**

In this cell, we calculate the depth of the quantum circuit using the `depth()` method. The depth of a quantum circuit is defined as the maximum number of quantum gates that are applied sequentially, one after the other. This metric is important for assessing the complexity and efficiency of the circuit, as lower depth is typically preferred for reducing decoherence and improving fidelity on noisy quantum devices.


In [ ]:
# Calculate the depth of the quantum circuit
circuit_depth = qc.depth()
circuit_depth
